# MOVIE RECOMMENDATION SYSTEM

In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
# Load the datasets
credits_df = pd.read_csv("credits.csv")
movies_df = pd.read_csv("movies.csv")

In [3]:
# Merge the datasets on 'title'
movies_df = movies_df.merge(credits_df, on='title')


In [4]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# Select relevant columns
movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]


In [6]:
# Drop rows with missing values
movies_df.dropna(inplace=True)


In [7]:
# Function to convert strings to lists
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [8]:
# Apply the conversion function to 'genres' and 'keywords' columns
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)


In [9]:
# Function to create a string of combined features
def combine_features(row):
    return ' '.join(row['genres']) + ' ' + ' '.join(row['keywords']) + ' ' + row['overview']


In [10]:
# Apply the function to create a new 'combined_features' column
movies_df['combined_features'] = movies_df.apply(combine_features, axis=1)


In [11]:
# Import CountVectorizer to convert text into a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer


In [12]:
# Initialize CountVectorizer
cv = CountVectorizer()


In [13]:
# Fit and transform 'combined_features' to a sparse matrix of token counts
count_matrix = cv.fit_transform(movies_df['combined_features'])

In [14]:
# Import cosine_similarity to calculate the similarity between movies
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
# Calculate the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix)


In [20]:
# Function to get movie recommendations
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = movies_df[movies_df['title'] == title].index[0]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_df['title'].iloc[movie_indices]

In [28]:
movies_df.head()

,movie_id,title,overview,genres,keywords,cast,crew,combined_features
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Action Adventure Fantasy Science Fiction cultu...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Adventure Fantasy Action ocean drug abuse exot...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Action Adventure Crime spy based on novel secr...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Action Crime Drama Thriller dc comics crime fi...
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",Action Adventure Science Fiction based on nove...


In [25]:
recommendations = get_recommendations('Avatar')
print(recommendations)

2409                 Aliens
1537              Moonraker
300       Starship Troopers
373         Mission to Mars
838                  Alien³
2021             Spaceballs
4               John Carter
582     Battle: Los Angeles
4337         Silent Running
3163                  Alien
Name: title, dtype: object
